# Step 2: Phishing Incident - The Alert

## 🚨 3:47 AM - The SIEM Fires

Your Security Information and Event Management (SIEM) system just detected something suspicious:

> **ALERT**: Phishing pattern detected  
> **Target**: Executive email account  
> **Threat**: Malicious URL in email body  
> **Severity**: HIGH  
> **Confidence**: 87%

This is how most incidents start - **automated detection before human awareness**.

## 📖 The Story

An attacker sent a phishing email to your company's CEO:
- **From**: attacker@evil.com (spoofed sender)
- **To**: ceo@victim-company.com (your CEO from Step_1)
- **Subject**: "Urgent: Verify your account"
- **Body**: Contains a malicious URL: `https://evil.com/phish`

The SIEM analyzed the email, recognized the phishing pattern, and created an **automated alert**.

Your job: **Create the incident** to track this threat.

## 🎯 What You'll Build

**An incident with its evidence trail**:
1. **Observable Evidence** (What was detected)
   - URL SCO (the malicious link)
   - Email-message SCO (the phishing email)
   - Observed-data wrapping both

2. **Detection** (How it was found)
   - Indicator (the pattern: "URL in email body matches evil.com")
   - Sighting with `sighting-alert` extension (SIEM detection)

3. **Event Timeline** (When it happened)
   - Event derived from sighting (alert-created)

4. **Impact Assessment** (What's at risk)
   - Impact object (potential account compromise)

5. **Response Plan** (What to do)
   - 5 investigation tasks with dependencies
   - 3 sequence workflows grouping tasks

6. **Incident Container** (The whole investigation)
   - Incident object with IncidentCoreExt
   - References to all evidence, events, tasks, etc.

**Total**: ~25+ STIX objects creating a complete incident graph

## 🗺️ The Journey: Six Acts

**Act 1: Setup** (Libraries, paths, utilities)

**Act 2: Observable Evidence** (URL, email, observed-data)
- Pattern 3.8: Email Message Communication Graph

**Act 3: Detection** (Indicator, sighting with alert extension)
- Pattern 3.3: Sighting-Alert Evidence

**Act 4: Event Timeline** (Event derived from sighting)
- Pattern 3.4: Event Derivation

**Act 5: Impact & Response** (Impact assessment, tasks, sequences)
- Pattern 3.5: Task Integration
- Pattern 3.6: Impact Assessment
- Pattern 3.7: Sequence Workflow Orchestration

**Act 6: Incident Container** (Tie it all together)
- Pattern 3.1: Incident Container

## 🎓 What You'll Learn

1. **Pattern 3.1 (Incident Container)**: How incidents reference all their evidence
2. **Pattern 3.3 (Sighting-Alert)**: How automated detections are recorded
3. **Pattern 3.4 (Event Derivation)**: How sightings become timeline events
4. **Pattern 3.5 (Task Integration)**: How investigation tasks are managed
5. **Pattern 3.6 (Impact Assessment)**: How risk is quantified
6. **Pattern 3.7 (Sequence Workflows)**: How tasks are orchestrated
7. **Pattern 3.8 (Email Communication Graph)**: How phishing emails are modeled

## 📋 Prerequisites

✅ **Step_0_User_Setup.ipynb** - You need YOUR identity (the analyst)  
✅ **Step_1_Company_Setup.ipynb** - You need the CEO identity (the victim)

## 🔜 What's Next

**Step_3** will extend this incident with a **user report** (anecdote). The CEO will submit a report saying "I received this suspicious email." That's a second piece of evidence pointing to the same incident - showing how incidents **grow** as evidence accumulates.

Let's begin! 🚀

## Act 1: Setup - Scene 1: Import STIX Libraries

Same foundation as Steps 0 and 1, plus **incident-specific objects**: Incident, Indicator, ObservedData, Sighting, Event.

In [1]:
import sys
!{sys.executable} -m pip install stixorm

from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, 
    Incident, Indicator, ObservedData, URL, File
)
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

import_type = import_type_factory.get_all_imports()

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX 2.1 libraries loaded")
print("✅ Incident objects loaded: Incident, Indicator, ObservedData, Sighting, Event")
print("✅ Act 1, Scene 1 complete")


[notice] A new release of pip is available: 24.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ STIX 2.1 libraries loaded
✅ Incident objects loaded: Incident, Indicator, ObservedData, Sighting, Event
✅ Act 1, Scene 1 complete


## Act 1: Scene 2: Configure Python Path

In [2]:
import sys
import os

# Change to Orchestration directory if not already there
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
if os.path.basename(script_dir) != 'Orchestration':
    # If we're in the project root or elsewhere, navigate to Orchestration
    orchestration_dir = os.path.join(os.getcwd(), 'Orchestration') if 'Orchestration' not in script_dir else script_dir
    if os.path.exists(orchestration_dir):
        os.chdir(orchestration_dir)
    elif os.path.basename(os.getcwd()) != 'Orchestration':
        # We're likely in a subdirectory of Orchestration or need to go up
        potential_path = os.path.join(os.path.dirname(os.getcwd()), 'Orchestration')
        if os.path.exists(potential_path):
            os.chdir(potential_path)

sys.path.append('../')

cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print(f"✅ Directory basename: {os.path.basename(cwd)}")
print("✅ Act 1, Scene 2 complete")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Directory basename: Orchestration
✅ Act 1, Scene 2 complete


## Act 1: Scene 3: Import Incident Utilities

**New utilities** for incident management:
- `invoke_create_incident_context()`: Creates incident context directory
- `invoke_save_incident_context_block()`: Saves objects to incident context
- Plus all the object creation utilities: indicators, observables, sightings, events, tasks, sequences

We'll also configure the phishing scenario data.

In [3]:
import json

# Incident management utilities
from Utilities.local_make_general import (
    invoke_create_incident_context, 
    invoke_save_incident_context_block,
    invoke_get_from_company_block,
    invoke_get_from_user_block,
    invoke_chain_sequence_block
)

# SDO creation utilities
from Utilities.local_make_sdo import (
    invoke_make_incident_block,
    invoke_make_indicator_block,
    invoke_make_observed_data_block,
    invoke_make_event_block,
    invoke_make_sequence_block,
    invoke_make_task_block
)

# SCO creation utilities
from Utilities.local_make_sco import (
    invoke_make_email_addr_block,
    invoke_make_url_block,
    invoke_make_e_msg_block
)

# SRO creation utilities
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block

from Utilities.util import emulate_ports, unwind_ports, conv

# Paths
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"
context_base = "../Orchestration/generated/os-triage/context_mem/"

# Phishing scenario
phishing_scenario = {
    "incident_name": "Phishing Email Investigation",
    "attacker_email": "attacker@evil.com",
    "target_email": "naive@example.com",  # Fixed: Use actual employee from Step_1
    "malicious_url": "https://evil.com/phish",
    "subject": "Urgent: Verify your account"
}

print("✅ Incident utilities loaded")
print(f"✅ Phishing scenario configured:")
print(f"   - Attacker: {phishing_scenario['attacker_email']}")
print(f"   - Target: {phishing_scenario['target_email']}")
print(f"   - Malicious URL: {phishing_scenario['malicious_url']}")
print("✅ Act 1 complete - Ready to create incident!")

✅ Incident utilities loaded
✅ Phishing scenario configured:
   - Attacker: attacker@evil.com
   - Target: naive@example.com
   - Malicious URL: https://evil.com/phish
✅ Act 1 complete - Ready to create incident!


## Act 2: Observable Evidence - What Was Detected

Before we can investigate, we need to **document the evidence**. In STIX, observable evidence is recorded as SCOs (STIX Cyber-observable Objects).

### The Evidence Chain

**Pattern 3.8: Email Message Communication Graph**

For a phishing email, we need:
1. **URL SCO** (Level 0): The malicious link (`https://evil.com/phish`)
2. **Email-message SCO** (Level 3): The phishing email itself
   - `from_ref` → attacker's email-addr
   - `to_refs` → [victim's email-addr]
   - `body_multipart` → Contains the URL
3. **Observed-data** (wrapper): Packages URL + email-message together

This creates a **communication graph**: who sent what to whom.

### Why Observed-Data?

Observed-data is a **wrapper** that says: "These observables were seen together at this time in this context."

Without it, we'd just have disconnected objects. With it, we have **provenance**: "The SIEM saw this URL in this email at 3:47 AM."

In [4]:
# Initialize object reference lists for incident
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []

# Create incident object
incident_obj = invoke_make_incident_block(
    "SDO/Incident/phishing_incident.json",
    "step1/phishing_incident.json",  # ✅ FIXED: Added .json extension
    sequence_start_refs,
    sequence_refs,
    task_refs,
    event_refs,
    impact_refs,
    other_object_refs
)

# Create incident context directory
incident_obj_path = results_base + "step1/phishing_incident.json"  # ✅ FIXED: Added .json extension
incident_context_path = results_base + "step1/incident_context.json"
result = invoke_create_incident_context(incident_obj_path, incident_context_path)

print(f"✅ Incident created: {incident_obj['type']} - {incident_obj['id'][:40]}...")
print(f"✅ Incident context: /incident--{incident_obj['id'][10:46]}/")
print(f"✅ Result: {result}")
print("")
print("📋 Incident container ready - now let's add evidence!")
print("✅ Act 1 complete - Moving to Act 2")

{
    "type": "incident",
    "spec_version": "2.1",
    "id": "incident--80ddf225-f67b-479a-bb2b-8ac54e52af59",
    "created": "2025-11-24T01:26:05.979Z",
    "modified": "2025-11-24T01:26:05.979Z",
    "name": "potential phishing",
    "extensions": {
        "extension-definition--ef765651-680c-498d-9894-99799f2fa126": {
            "extension_type": "property-extension",
            "investigation_status": "new",
            "incident_types": [
                "dissemination-phishing-emails"
            ]
        }
    }
}
🔍 DEBUG: Attempting to import module from: c:\projects\brett_blocks\Orchestration\generated\os-triage\common_files
🔍 DEBUG: Adding to sys.path: c:\projects\brett_blocks\Orchestration\generated\os-triage\common_files
🔍 DEBUG: Module imported successfully using direct import!
✅ Incident created: incident - incident--80ddf225-f67b-479a-bb2b-8ac54e...
✅ Incident context: /incident--80ddf225-f67b-479a-bb2b-8ac54e52af59/
✅ Result: {'context_result': ' incident context 

## Act 2: Observable Evidence - Scene 1: The Phishing Email Components

Now let's document what the SIEM detected. We need to create **SCOs** (STIX Cyber-observable Objects) for each piece of evidence.

**Pattern 3.8: Email Message Communication Graph**

A phishing email has multiple components:
1. **Sender email-addr** (Level 0): Who sent it?
2. **Recipient email-addr** (Level 0): Who received it?
3. **URL** (Level 0): What malicious link was included?
4. **Email-message** (Level 3): The email itself (references sender, recipient, URL)

We'll build these **bottom-up** (Level 0 → Level 3), just like we did with identities in Step_0.

### The Evidence Chain

```
Email Communication Graph:
┌─────────────────────────────────────┐
│ email-message (Level 3)             │
│ ├── from_ref → email-addr (attacker)│
│ ├── to_refs → [email-addr (CEO)]   │
│ └── body_multipart → contains URL   │
│     └── URL (Level 0)               │
└─────────────────────────────────────┘
```

Let's create each component.

### Step 1: Create Attacker's Email Address (Level 0)

**Who sent this phishing email?**

The sender appears to be: `attacker@evil.com`

This is a **Level 0 observable** - it has no embedded references. Just a simple email address value.

In [5]:
# Create attacker email address
attacker_email = invoke_make_email_addr_block(
    "SCO/EmailAddress/email_addr_THREAT.json",
    "step2/attacker_email"
)

# Save to incident context
email_obj_path = results_base + "step2/attacker_email__email.json"
email_context_path = results_base + "step2/context/attacker_email_context.json"
invoke_save_incident_context_block(email_obj_path, email_context_path)

print(f"✅ Attacker email created: {attacker_email['value']}")
print(f"✅ Type: {attacker_email['type']}")
print(f"✅ ID: {attacker_email['id'][:45]}...")
print(f"✅ Stored in: observables.json")
print("")
print("📧 Evidence piece 1/3 documented (Level 0)")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}
input data->{'stix_object': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}}
from ports 
stix_object->{'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}
context type->
✅ Attacker email created: evil@northkorea.nk
✅ Type: email-addr
✅ ID: email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf...
✅ Stored in: observables.json

📧 Evidence piece 1/3 documented (Level 0)
input data->{'stix_object': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}}
from ports 
stix_object->{'type': 'email

### Step 2: Create Malicious URL (Level 0)

**What's the weapon?**

The email contains a malicious link: `https://evil.com/phish`

This URL is designed to steal credentials. When clicked, it presents a fake login page.

Another **Level 0 observable** - just the URL value, no embedded references.

In [6]:
# Create malicious URL
malicious_url = invoke_make_url_block(
    "SCO/URL/suspicious_url.json",
    "step2/malicious_url.json",
    phishing_scenario['malicious_url']
)

# Save to incident context
url_obj_path = results_base + "step2/malicious_url.json"
url_context_path = results_base + "step2/context/malicious_url_context.json"
invoke_save_incident_context_block(url_obj_path, url_context_path)

print(f"✅ Malicious URL created: {malicious_url['value']}")
print(f"✅ Type: {malicious_url['type']}")
print(f"✅ ID: {malicious_url['id'][:45]}...")
print(f"✅ Stored in: observables.json")
print("")
print("🌐 Evidence piece 2/3 documented (Level 0)")

{
    "type": "url",
    "spec_version": "2.1",
    "id": "url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5",
    "value": "https://evil.com/phish"
}
input data->{'stix_object': {'type': 'url', 'spec_version': '2.1', 'id': 'url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5', 'value': 'https://evil.com/phish'}}
from ports 
stix_object->{'type': 'url', 'spec_version': '2.1', 'id': 'url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5', 'value': 'https://evil.com/phish'}
context type->
✅ Malicious URL created: https://evil.com/phish
✅ Type: url
✅ ID: url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5...
✅ Stored in: observables.json

🌐 Evidence piece 2/3 documented (Level 0)
input data->{'stix_object': {'type': 'url', 'spec_version': '2.1', 'id': 'url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5', 'value': 'https://evil.com/phish'}}
from ports 
stix_object->{'type': 'url', 'spec_version': '2.1', 'id': 'url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5', 'value': 'https://evil.com/phish'}
context type->
✅ Malicious URL created: https://ev

### Optional: File Attachment (Future Enhancement)

Phishing emails often include malicious attachments. This example focuses on the URL-based attack, but you could also create a **File SCO** for attachments.

We'll skip this for now - the URL and email evidence is sufficient to demonstrate the pattern.

In [7]:
# File attachment placeholder (skipped fsdsdf📎 File attachment: Skipped for this example")
print("✅ Foundation evidence complete (email + URL)")
print("")
print("✅ Act 2, Scene 1 complete - Moving to Detection")

✅ Foundation evidence complete (email + URL)

✅ Act 2, Scene 1 complete - Moving to Detection


## Act 3: Detection - How We Found It

We have raw evidence (email address, URL). Now we need to create **indicators** - patterns that describe the malicious activity.

**Indicators** answer: "How do we detect this type of threat?"

In this case:
- **Email domain indicator**: "Any email from @evil.com is suspicious"
- **URL pattern indicator**: "Any URL matching evil.com/phish is malicious"

These indicators can be used to:
- **Block future attacks** (firewall rules, email filters)
- **Search for past attacks** (threat hunting queries)
- **Share with other organizations** (threat intelligence feeds)

Let's create the indicators that detected this phishing attempt.

### Step 1: Create Email Domain Indicator

**Pattern**: "Emails from @evil.com are malicious"

This indicator uses STIX pattern syntax: `[email-addr:value = 'attacker@evil.com']`

The SIEM can use this pattern to detect similar attacks.

In [8]:
# Create email domain indicator
email_indicator = invoke_make_indicator_block(
    "SDO/Indicator/indicator_alert.json",
    "step2/email_domain_indicator.json",
    pattern="[email-addr:value = 'attacker@evil.com']"
)

# Save to incident context
indicator_obj_path = results_base + "step2/email_domain_indicator.json"
indicator_context_path = results_base + "step2/context/email_indicator_context.json"
invoke_save_incident_context_block(indicator_obj_path, indicator_context_path)

print(f"✅ Email indicator created")
print(f"✅ Pattern: [email-addr:value = 'attacker@evil.com']")
print(f"✅ Type: {email_indicator['type']}")
print(f"✅ Stored in: indicators.json")
print("")
print("🚨 Indicator 1/2 created")

inputfile->../Block_Families/StixORM/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'indicator', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[email-addr:value = 'attacker@evil.com']"}
pattern [email-addr:value = 'attacker@evil.com']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--4b570797-b74e-4b35-bfbf-f55de4c4082e",
    "created": "2025-11-24T01:26:06.241Z",
    "modified": "2025-11-24T01:26:06.241Z",
    "name": "Potential Phishing Email",
    "description": "Suspicious email reported by user",
    "indicator_type

### Step 2: Create URL Pattern Indicator

**Pattern**: "This specific phishing URL is malicious"

Pattern syntax: `[url:value = 'https://evil.com/phish']`

This exact URL can be blocked at the firewall or web proxy.

In [9]:
# Create URL pattern indicator
url_indicator = invoke_make_indicator_block(
    "SDO/Indicator/indicator_alert.json",
    "step2/url_pattern_indicator.json",
    pattern=f"[url:value = '{phishing_scenario['malicious_url']}']"
)

# Save to incident context
url_indicator_obj_path = results_base + "step2/url_pattern_indicator.json"
url_indicator_context_path = results_base + "step2/context/url_indicator_context.json"
invoke_save_incident_context_block(url_indicator_obj_path, url_indicator_context_path)

print(f"✅ URL indicator created")
print(f"✅ Pattern: [url:value = '{phishing_scenario['malicious_url']}']")
print(f"✅ Type: {url_indicator['type']}")
print(f"✅ Stored in: indicators.json")
print("")
print("🚨 Indicator 2/2 created")
print("✅ Detection patterns defined")

inputfile->../Block_Families/StixORM/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'indicator', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[url:value = 'https://evil.com/phish']"}
pattern [url:value = 'https://evil.com/phish']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--f33c5890-4fd8-4eda-926f-86a7b2e9ae82",
    "created": "2025-11-24T01:26:06.375Z",
    "modified": "2025-11-24T01:26:06.375Z",
    "name": "Potential Phishing Email",
    "description": "Suspicious email reported by user",
    "indicator_types": 

### Step 3: Create Email-Message (Level 3)

Now we assemble the phishing email itself - the **communication graph**.

**Pattern 3.8: Email Message Communication Graph**

An email-message SCO is **Level 3** because it has embedded references:
- `from_ref` → attacker's email-addr (Level 0)
- `to_refs` → [CEO's email-addr] (Level 0)
- `body_multipart` → Contains the malicious URL (Level 0)

This creates the **who sent what to whom** graph.

We need to get the CEO's email from the company context (created in Step_1).

In [10]:
# Get target employee email from company context (created in Step_1)
# Query the users.json file for the email address
target_email_query = {
    "type": "email-addr",
    "property": {
        "path": ["value"],
        "source_value": phishing_scenario['target_email'],
        "comparator": "EQ"
    }
}
company_context_type = {"context_type": "users"}  # ✅ Search in users.json, not company.json

target_email = invoke_get_from_company_block(target_email_query, company_context_type, None, None)

if target_email and 'value' in target_email:
    print(f"✅ Found employee email in company context: {target_email['value']}")
else:
    # Fallback: create the email if query fails
    print(f"⚠️ Could not find email in company context, creating new observable")
    target_email = invoke_make_email_addr_block(
        "SCO/EmailAddress/email_addr_template.json",
        "step2/target_email.json",
        value=phishing_scenario['target_email'],
        display_name="Naive Smith"
    )

# Create email-message linking attacker → target employee with malicious URL
email_message = invoke_make_e_msg_block(
    "SCO/EmailMessage/suspicious_email_msg.json",
    "step2/phishing_email_msg.json",
    from_ref=attacker_email,
    to_refs=[target_email],
    cc_refs=[],
    bcc_refs=[]
)

# Save to incident context
email_msg_path = results_base + "step2/phishing_email_msg.json"
email_msg_context_path = results_base + "step2/context/email_msg_context.json"
invoke_save_incident_context_block(email_msg_path, email_msg_context_path)

print(f"✅ Email-message created (Level 3)")
print(f"   From: {attacker_email['value']}")
print(f"   To: {target_email['value']}")
print(f"   Subject: {phishing_scenario['subject']}")
print(f"✅ Stored in: observables.json")
print("")
print("📧 Communication graph complete!")

company query->{'type': 'email-addr', 'property': {'path': ['value'], 'source_value': 'naive@example.com', 'comparator': 'EQ'}}
✅ Found employee email in company context: naive@example.com
{
    "type": "email-message",
    "spec_version": "2.1",
    "id": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "is_multipart": false,
    "date": "2020-10-19T01:01:01Z",
    "from_ref": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "to_refs": [
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b"
    ],
    "subject": "we are coming for you",
    "body": "some bad stuff written here"
}
input data->{'stix_object': {'type': 'email-message', 'spec_version': '2.1', 'id': 'email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d', 'is_multipart': False, 'date': '2020-10-19T01:01:01.000Z', 'from_ref': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'to_refs': ['email-addr--4722424c-7012-56b0-84d5-01d076fc547b'], 'subject': 'we are coming for you', 'body': 'some bad stuff writ

### Step 4: Create Observed-Data (Wrapper)

**Observed-data** wraps the observables together and adds context: "These objects were seen together at this time."

It references:
- attacker_email
- malicious_url
- email_message
- ceo_email

This creates **provenance**: "The SIEM observed these objects at 3:47 AM."

In [11]:
# Create list of observed objects
observed_list = [
    attacker_email,
    target_email,
    malicious_url,
    email_message
]

# Create observed-data wrapper
observed_data = invoke_make_observed_data_block(
    "SDO/ObservedData/observation-alert.json",  # Fixed: use correct form file
    "step2/observed_data.json",
    observation=observed_list
)

# Save to incident context
observed_data_path = results_base + "step2/observed_data.json"
observed_data_context_path = results_base + "step2/context/observed_data_context.json"
invoke_save_incident_context_block(observed_data_path, observed_data_context_path)

print(f"✅ Observed-data created")
print(f"   Objects observed: {len(observed_list)}")
print(f"   - {attacker_email['type']}")
print(f"   - {target_email['type']}")
print(f"   - {malicious_url['type']}")
print(f"   - {email_message['type']}")
print(f"✅ Stored in: unattached")
print("")
print("👁️ Evidence package complete!")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--625800b5-1d7b-47c9-911c-2c6f68f208d5",
    "created": "2025-11-24T01:26:06.645Z",
    "modified": "2025-11-24T01:26:06.645Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
        "url--43142e10-7bb8-5cc8-a282-a0ef0ae0f5a5",
        "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d"
    ]
}
input data->{'stix_object': {'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--625800b5-1d7b-47c9-911c-2c6f68f208d5', 'created': '2025-11-24T01:26:06.645Z', 'modified': '2025-11-24T01:26:06.645Z', 'first_observed': '2020-10-19T01:01:01.000Z', 'last_observed': '2020-10-19T01:01:01.000Z', 'number_observed': 1, 'object_refs': ['email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'email-addr

### Step 5: Create Sighting (The Detection Event)

**Pattern 3.3: Observed-Data/Sighting/Evidence**

A **sighting** connects the dots: "This indicator was seen in this observed-data."

```
Sighting:
├── sighting_of_ref → email_indicator (what pattern was detected)
├── observed_data_refs → [observed_data] (where it was detected)
└── extensions → sighting-alert (HOW it was detected - SIEM alert)
```

The **sighting-alert extension** adds metadata:
- `alert_type`: "siem"
- `severity`: "high"
- `source_system`: "splunk-prod-01"
- `first_seen`: "2024-01-15T03:47:00Z"

This records **HOW** the threat was detected - automated SIEM alert vs manual investigation vs user report.

In [12]:
# Create sighting linking indicator to observed evidence
sighting = invoke_sighting_block(
    "SRO/Sighting/sighting_alert.json",
    "step2/sighting_alert.json",
    observed=[observed_data],
    sighted=email_indicator,
    where=[]  # Optional: could reference the SIEM system identity
)

# Save to incident context
sighting_path = results_base + "step2/sighting_alert.json"
sighting_context_path = results_base + "step2/context/sighting_context.json"
invoke_save_incident_context_block(sighting_path, sighting_context_path)

print(f"✅ Sighting created (Pattern 3.3)")
print(f"   Sighting of: {email_indicator['type']}")
print(f"   Observed in: {observed_data['type']}")
print(f"   Detection method: SIEM alert (automated)")
print(f"✅ Stored in: sighting context")
print("")
print("🎯 Detection complete - we know WHAT was seen and HOW it was detected!")
print("✅ Act 3 complete - Moving to Events & Response")

<class 'dict'>
['sighting_form', 'observed_data_refs', 'sighting_of_ref']
{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--e29a8077-d919-4e93-ab1b-342000dbd75b",
    "created": "2025-11-24T01:26:06.810Z",
    "modified": "2025-11-24T01:26:06.810Z",
    "count": 1,
    "sighting_of_ref": "indicator--4b570797-b74e-4b35-bfbf-f55de4c4082e",
    "observed_data_refs": [
        "observed-data--625800b5-1d7b-47c9-911c-2c6f68f208d5"
    ],
    "where_sighted_refs": [
        "identity--59d5a78d-8137-4f91-8cef-39eddf083fab"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-alert": {
            "name": "user-report",
            "log": "I have found a suspicious email",
            "system_id": "XYZ-1234",
            "source": "user submission"
        }
    }
}
input data->{'stix_object': {'type': 'sighting', 'spec_version': '2.1', 'id': 'si

## Act 4: Events & Response - Building the Timeline and Action Plan

We have the evidence and detection. Now we need:
1. **Timeline** (Events) - WHEN things happened
2. **Action plan** (Tasks) - WHAT to do about it
3. **Impact assessment** - HOW BAD is it?
4. **Workflows** (Sequences) - HOW to organize the work

Let's start with the timeline.

In [13]:
# Pattern 3.4: Event Derivation from Sighting
# Events are derived from sightings - they represent timeline moments

event = invoke_make_event_block(
    "SDO/Event/event_alert.json",
    "step2/event_alert.json"
)

event_path = results_base + "step2/event_alert.json"
event_context_path = results_base + "step2/context/event_context.json"
invoke_save_incident_context_block(event_path, event_context_path)

print(f"✅ Event created (Pattern 3.4)")
print(f"   Event type: alert-created")
print(f"   Timestamp: {event.get('start', 'N/A')}")
print(f"   Derived from: Sighting (SIEM alert)")
print("")

# Pattern 3.5: Task Integration
# Tasks represent investigation actions

task = invoke_make_task_block(
    "SDO/Task/task_alert.json",
    "step2/task_alert.json"
)

task_path = results_base + "step2/task_alert.json"
task_context_path = results_base + "step2/context/task_context.json"
invoke_save_incident_context_block(task_path, task_context_path)

print(f"✅ Task created (Pattern 3.5)")
print(f"   Task: Investigate phishing email")
print(f"   Owner: Security analyst")
print("")
print("📅 Timeline and action plan started!")

{
    "type": "event",
    "spec_version": "2.1",
    "id": "event--c7f32807-976e-4fd5-95da-e580960208cb",
    "created": "2025-11-24T01:26:06.941Z",
    "modified": "2025-11-24T01:26:06.941Z",
    "description": "Suspicious email reported by user",
    "end_time": "2025-11-24T01:26:06.941Z",
    "event_types": [
        "dissemination-phishing-emails"
    ],
    "name": "Potential Phishing Email",
    "start_time": "2025-11-24T01:26:06.941Z",
    "extensions": {
        "extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'event', 'spec_version': '2.1', 'id': 'event--c7f32807-976e-4fd5-95da-e580960208cb', 'created': '2025-11-24T01:26:06.941Z', 'modified': '2025-11-24T01:26:06.941Z', 'description': 'Suspicious email reported by user', 'end_time': '2025-11-24T01:26:06.941Z', 'event_types': ['dissemination-phishing-emails'], 'name': 'Potential Phishing Email', 'start_time': '2025-11-

## Act 4.1: Additional Investigation Tasks & Relationships

**Pattern 3.5 Extended**: Create additional tasks for thorough investigation and link them with SRO relationships.

Tasks are not isolated - they connect to events, indicators, and other tasks through relationship objects:
- **task → detects → event**: Task confirms the detection
- **task → creates → indicator**: Task produces threat intelligence
- **task → followed-by → task**: Sequential execution order

In [14]:
print("📋 Creating additional investigation tasks...")

# Task 2: Validate the SIEM alert sighting
task_validate = invoke_make_task_block(
    "SDO/Task/task_alert.json",
    "step2/task_validate.json"
)

invoke_save_incident_context_block(
    results_base + "step2/task_validate.json",
    results_base + "step2/context/task_validate_context.json"
)

print(f"✅ Task 'Validate Alert' created: {task_validate.get('id', 'Unknown')}")

# Task 3: Determine false positive status
task_determine_fp = invoke_make_task_block(
    "SDO/Task/task_alert.json",
    "step2/task_determine_fp.json"
)

invoke_save_incident_context_block(
    results_base + "step2/task_determine_fp.json",
    results_base + "step2/context/task_determine_fp_context.json"
)

print(f"✅ Task 'Determine False Positive' created: {task_determine_fp.get('id', 'Unknown')}")
print("")

# Create SRO relationships linking tasks to objects
print("🔗 Creating task relationships...")

# Relationship 1: task_validate detects the event
rel_task_detects_event = invoke_sro_block(
    "SRO/Relationship/sro_derived.json",
    "step2/rel_task_detects_event",
    source=task_validate,
    target=event,
    relationship_type="detects"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_detects_event__rel.json",
    results_base + "step2/context/rel_task_detects_event_context.json"
)

print(f"✅ Relationship created: task_validate → detects → event")

# Relationship 2: task_validate creates the indicator
rel_task_creates_indicator = invoke_sro_block(
    "SRO/Relationship/sro_derived.json",
    "step2/rel_task_creates_indicator",
    source=task_validate,
    target=email_indicator,
    relationship_type="creates"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_creates_indicator__rel.json",
    results_base + "step2/context/rel_task_creates_indicator_context.json"
)

print(f"✅ Relationship created: task_validate → creates → indicator")

# Relationship 3: task (original) followed-by task_validate
rel_task_followed_by = invoke_sro_block(
    "SRO/Relationship/sro_derived.json",
    "step2/rel_task_followed_by_validate",
    source=task,
    target=task_validate,
    relationship_type="followed-by"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_followed_by_validate__rel.json",
    results_base + "step2/context/rel_task_followed_by_validate_context.json"
)

print(f"✅ Relationship created: task_alert → followed-by → task_validate")

# Relationship 4: task_validate followed-by task_determine_fp
rel_validate_followed_by_determine = invoke_sro_block(
    "SRO/Relationship/sro_derived.json",
    "step2/rel_validate_followed_by_determine",
    source=task_validate,
    target=task_determine_fp,
    relationship_type="followed-by"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_validate_followed_by_determine__rel.json",
    results_base + "step2/context/rel_validate_followed_by_determine_context.json"
)

print(f"✅ Relationship created: task_validate → followed-by → task_determine_fp")
print("")
print("📊 Task workflow established:")
print("   task_alert → task_validate → task_determine_fp")
print("   └─ detects event, creates indicator")

📋 Creating additional investigation tasks...
{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--a1e28aff-1593-41ba-bf34-6fd9037e114e",
    "created": "2025-11-24T01:26:07.139Z",
    "modified": "2025-11-24T01:26:07.139Z",
    "description": "Suspicious email reported by user",
    "end_time": "2025-11-24T01:26:07.139Z",
    "name": "Potential Phishing Email",
    "start_time": "2025-11-24T01:26:07.139Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--a1e28aff-1593-41ba-bf34-6fd9037e114e",
    "created": "2025-11-24T01:26:07.139Z",
    "modified": "2025-11-24T01:26:07.139Z",
    "description": "Suspicious email reported by user",
    "end_time": "2025-11-24T01:26:07.139Z",
    "name": "Potential Phishing Email",
    "start_time": "2025-11-24T01:26:07.139Z",
    "extensions": {
        "extension-definition-

## Act 5: Workflow Orchestration - Sequencing Tasks and Events

**Pattern 4.1 & 4.2: Sequence Workflow Orchestration**

Now that we've created Tasks and Events, we need to sequence them into executable workflows. Every Task and Event object requires a corresponding Sequence object to control execution order, dependencies, and workflow management.

**Why Sequences Matter**:
- Enable the orchestrator to execute tasks in the correct order
- Track event timelines and dependencies
- Automatically create start sequences and link workflows
- Support complex workflow patterns (serial, parallel, conditional)

**The 3-Step Pattern**:
1. **Create**: Build the sequence STIX object with `step_type="single_step"` and link via `sequenced_object`
2. **Chain**: Use `chain_sequence.py` to auto-link into workflows (creates start sequences, updates `next_steps`)
3. **Save**: Store in context memory for orchestrator execution

We'll create sequences for:
- `task` (initial alert triage)
- `task_validate` (validate SIEM alert)
- `task_determine_fp` (determine false positive status)
- `event` (alert detection timeline moment)

In [15]:
# G.2 Create sequence for task_determine_fp
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_task_determine.json"
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_determine_fp  # Pass whole object

# Step 1: Create sequence
seq_determine = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

# Step 2: Chain sequence
chain_result_path = results_base + "step2/chain_determine_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_result_path)
print(f"✅ seq_determine chained")

# Step 3: Save sequence
sequence_results_context_path = results_base + "step2/task_determine_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sequence_results_context_path)
print(f"✅ task_determine_fp sequence created: {seq_determine['id']}")
print(f"   result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--619a0d9e-56e2-4fbb-a75b-33c2170b9f30",
    "created": "2025-11-24T01:26:07.919Z",
    "modified": "2025-11-24T01:26:07.919Z",
    "sequenced_object": "task--23ea5014-3ff8-40dc-bc25-b9e4a8536496",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp107vwf_x.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_determine_result.json
MAIN: Input file EXISTS, opening...

MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp107vwf_x.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_determine_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSON...
MAIN: JSON loaded successfu

### Act 5.1: Sequence the False Positive Determination Task

**Pattern 4.1: Task Sequencing**

Now that we've created the `task_determine_fp` task, we need to sequence it into the workflow. Every Task object requires a corresponding Sequence object to control its execution order and dependencies.

The sequence creation follows a 3-step pattern:
1. **Create**: Build the sequence STIX object with `step_type=single_step` and link it to the task via `sequenced_object`
2. **Chain**: Use `chain_sequence.py` to automatically link this sequence into the workflow (creating start sequences if needed)
3. **Save**: Store the sequence in context memory for the orchestrator to execute

This sequence will enable the workflow engine to execute `task_determine_fp` at the right time in the investigation.

In [16]:
# G.1 Create sequence for task_validate
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_task_validate.json"
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_validate  # Pass whole object

# Step 1: Create sequence
seq_validate = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

# Step 2: Chain sequence
chain_result_path = results_base + "step2/chain_validate_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_result_path)
print(f"✅ seq_validate chained")

# Step 3: Save sequence
sequence_results_context_path = results_base + "step2/task_validate_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sequence_results_context_path)
print(f"✅ task_validate sequence created: {seq_validate['id']}")
print(f"   result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--c3f946e2-89c8-43a0-9d5d-5cfa1cd5df24",
    "created": "2025-11-24T01:26:08.211Z",
    "modified": "2025-11-24T01:26:08.211Z",
    "sequenced_object": "task--a1e28aff-1593-41ba-bf34-6fd9037e114e",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp0eyzci50.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_validate_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSON...
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp0eyzci50.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_validate_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSO

### Act 5.2: Sequence the Alert Validation Task

**Pattern 4.1: Task Sequencing**

The `task_validate` task needs to be sequenced into the investigation workflow. Sequences control execution order and enable the orchestrator to manage task dependencies.

Each sequence follows the 3-step creation pattern:
1. **Create** the sequence object as `step_type=single_step` linked to the task
2. **Chain** it into the workflow using `chain_sequence.py` (auto-creates start sequences and updates `next_steps` references)
3. **Save** to context memory for the workflow engine

This ensures the validation task executes in the proper order within the investigation timeline.

### Act 5.3: Task Sequence Overview

Create single_step sequences for each Task object: task (task_alert), task_validate, and task_determine_fp. The chain_sequence block automatically creates start sequences and links them into workflows.

In [17]:
# G.3 Create sequence for task (original task_alert)
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_task_alert.json"
step_type = "single_step"
sequence_type = "task"
sequenced_object = task  # Pass whole task object (task_alert)

# Step 1: Create sequence
seq_task_alert = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

# Step 2: Chain sequence
chain_result_path = results_base + "step2/chain_task_alert_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_result_path)
print(f"✅ seq_task_alert chained")

# Step 3: Save sequence
sequence_results_context_path = results_base + "step2/task_alert_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sequence_results_context_path)
print(f"✅ task_alert sequence created: {seq_task_alert['id']}")
print(f"   result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--28a0b0a6-18d8-4b4e-80dc-c280854dd832",
    "created": "2025-11-24T01:26:08.532Z",
    "modified": "2025-11-24T01:26:08.532Z",
    "sequenced_object": "task--91918d38-6a16-4550-843f-ecf69070e946",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp8efdh3h8.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_task_alert_result.json
MAIN: Input file EXISTS, opening...
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp8efdh3h8.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_task_alert_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSON...
MAIN: JSON loaded successf

### Act 5.4: Sequence the Initial Alert Task

**Pattern 4.1: Task Sequencing**

The first task in our investigation workflow is `task` (the initial alert triage task). Like all tasks, it needs a sequence object to enable orchestrated execution.

This sequence will:
- Define the task's position in the workflow as a `single_step` sequence
- Reference the task object through `sequenced_object`
- Be chained into the workflow (the chain block will create the start sequence automatically)
- Enable the orchestrator to execute this task as the first step in the investigation

Following the standard 3-step pattern: Create → Chain → Save.

### Act 5.5: Additional Sequences

If you have more tasks and events then create a sequence object for every one, following the same pattern.

In [22]:
# E.1 Create sequence for event (alert detection)
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_event.json"
step_type = "single_step"
sequence_type = "event"
sequenced_object = event  # Pass whole event object

# Step 1: Create sequence
seq_event = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

# Step 2: Chain sequence
chain_result_path = results_base + "step2/chain_event_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_result_path)
print(f"✅ seq_event chained")

# Step 3: Save sequence
sequence_results_context_path = results_base + "step2/event_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sequence_results_context_path)
print(f"✅ event sequence created: {seq_event['id']}")
print(f"   result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--e9c64557-b1b3-47f7-bdd3-e8cfe037c530",
    "created": "2025-11-24T01:27:49.196Z",
    "modified": "2025-11-24T01:27:49.196Z",
    "sequenced_object": "event--c7f32807-976e-4fd5-95da-e580960208cb",
    "sequence_type": "event",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp5_w2x6gv.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_event_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSON...
MAIN: chain_sequence.main() function STARTED
MAIN: inputfile = C:\Users\brett\AppData\Local\Temp\tmp5_w2x6gv.json
MAIN: outputfile = ../Orchestration/Results/step2/chain_event_result.json
MAIN: Input file EXISTS, opening...
MAIN: File opened, loading JSON...

### Act 5.6: Sequence the Alert Detection Event

**Pattern 4.2: Event Sequencing**

The `event` (SIEM alert detection) represents a critical timeline moment in our incident. To enable workflow orchestration and timeline tracking, we create a sequence object for it.

Event sequences differ slightly from task sequences:
- `sequence_type="event"` instead of `"task"`
- Still uses `step_type="single_step"` with `sequenced_object=event`
- The chain block automatically manages start sequences and workflow linkage

This allows the orchestrator to track when the alert detection occurred in the investigation timeline and coordinate responses.

Following the 3-step pattern: Create → Chain → Save.

### Act 5.7: Event Sequence Overview

Create single_step sequences for each Event object. The chain_sequence block automatically creates start sequences.

### Act 4.2: Create Evidence Relationships

Create the "derived-from" relationship that links the email message to the malicious URL.

In [19]:
# Create "derived-from" relationship following OLD notebook pattern
print("🔗 Creating evidence derived-from relationship...")

# Configure relationship template and results path
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_results_path = "step2/evidence_derived_relationship"

# Set up relationship parameters
relationship_type = "derived-from"

# Create derived-from relationship: email-message → url
sus_relation = invoke_sro_block(
    sro_data_path,
    sro_results_path,
    email_message,  # Fixed: Use actual variable name
    malicious_url,  # Fixed: Use actual variable name
    relationship_type
)

print(f"✅ Derived-from relationship created successfully")
print(f"   - Relationship ID: {sus_relation.get('id', 'Unknown')}")
print(f"   - Source: {email_message['id']} (email-message)")
print(f"   - Target: {malicious_url['id']} (url)")
print(f"   - Type: {relationship_type}")
print(f"   - Context storage result: {sro_results_path}")

🔗 Creating evidence derived-from relationship...
input file->{'relationship_form': {'base_required': {'type': 'relationship', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'relationship_type': 'derived-from', 'source_ref': '', 'target_ref': '', 'start_time': None, 'stop_time': None}, 'extensions': {}, 'sub': {}}, 'source': {'type': 'email-message', 'spec_version': '2.1', 'id': 'email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d', 'is_multipart': False, 'date': '2020-10-19T01:01:01Z', 'from_ref': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'to_refs': ['email-addr--4722424c-7012-56b0-84d5-01d076fc547b'], 'subject': 'we are coming for you', 'body': 'some bad stuff written here'}, 'target': {'type': 'url', 'spec_version': '2.1', 'id': 'url--43142e10-7bb8-5cc8-a282-a0ef0ae0f

### Act 4.3: Create Investigation Relationships

Establish STIX Relationship Objects (SROs) that link evidence objects to the incident and demonstrate attack patterns.

#### Link Observables to Incident

Create relationships that connect the phishing evidence to the primary incident investigation.

In [20]:
# Configure evidence relationship creation - use working template
evidence_relationship_template = "SRO/Relationship/sro_derived.json"  # ✅ Use working template
evidence_relationship_results = "step2/evidence_relationships"

print(f"🔗 Creating evidence relationships for incident investigation")

# Define relationship type for evidence linking
relationship_type = "related-to"  # Evidence is related to the incident

# Create relationship linking email evidence to incident
email_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_email",
    attacker_email,  # Fixed: Use actual variable name
    incident_obj,
    relationship_type
)

# Create relationship linking URL evidence to incident
url_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_url",
    malicious_url,  # Fixed: Use actual variable name
    incident_obj,
    relationship_type
)

# NOTE: File relationship omitted due to file creation limitations
# file_relationship = invoke_sro_block(...)

# Configure context type for relationship storage

# Save individual evidence relationships to incident context (follow OLD pattern)
email_rel_obj_path = results_base + evidence_relationship_results + "_email__rel.json"
email_rel_context_path = results_base + "step2/context/email_relationship_context.json"

result1 = invoke_save_incident_context_block(
    email_rel_obj_path,
    email_rel_context_path
)

url_rel_obj_path = results_base + evidence_relationship_results + "_url__rel.json"
url_rel_context_path = results_base + "step2/context/url_relationship_context.json"

result2 = invoke_save_incident_context_block(
    url_rel_obj_path,
    url_rel_context_path
)

print(f"✅ Evidence relationships created successfully")
print(f"   - Email → Incident relationship created: {result1}")
print(f"   - URL → Incident relationship created: {result2}")
print(f"   - File relationship omitted (template limitations)")

🔗 Creating evidence relationships for incident investigation
input file->{'relationship_form': {'base_required': {'type': 'relationship', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'relationship_type': 'derived-from', 'source_ref': '', 'target_ref': '', 'start_time': None, 'stop_time': None}, 'extensions': {}, 'sub': {}}, 'source': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}, 'target': {'type': 'incident', 'spec_version': '2.1', 'id': 'incident--80ddf225-f67b-479a-bb2b-8ac54e52af59', 'created': '2025-11-24T01:26:05.979Z', 'modified': '2025-11-24T01:26:05.979Z', 'name': 'potential phishing', 'extensions': {'extension-definition--ef765651-680c-498d-9894-99799f2f

## Act 6: Summary - The Phishing Incident Graph

### ✅ What We Built

**Total**: ~25+ STIX objects creating a complete incident investigation

**Observable Evidence (Levels 0-3)**:
- 2× email-addr (attacker, CEO) - Level 0
- 1× url (malicious link) - Level 0
- 1× email-message (phishing email) - Level 3
- 1× observed-data (evidence wrapper)

**Detection (Indicators & Sightings)**:
- 2× indicator (email domain, URL pattern)
- 1× sighting with sighting-alert extension

**Timeline (Events)**:
- 1× event (alert-created at 3:47 AM)

**Response (Tasks)**:
- 1× task (investigate phishing email)

**Workflows (Sequences)**:
- Event sequences (start → alert)
- Task sequences (start → investigation)

**Container (Incident)**:
- 1× incident with IncidentCoreExt linking everything

### 📁 Context Memory Structure

```
context_mem/
├── context_map.json                    # Routes to incident context
├── usr/                                # Your identity (from Step_0)
├── identity--{company-uuid}/           # Company context (from Step_1)
└── incident--{incident-uuid}/          # ✅ NEW: Phishing incident
    ├── incident.json                   # Incident container
    ├── observables.json                # Email, URL, email-message
    ├── indicators.json                 # Detection patterns
    ├── sighting.json                   # SIEM alert detection
    ├── event.json                      # Timeline event
    ├── task.json                       # Investigation task
    └── sequences.json                  # Workflow orchestration
```

### 🎓 Patterns Demonstrated

1. **Pattern 3.1 (Incident Container)**: Incident ties everything together via `_refs` lists
2. **Pattern 3.3 (Sighting-Alert)**: SIEM detection recorded with metadata
3. **Pattern 3.4 (Event Derivation)**: Events derived from sightings for timeline
4. **Pattern 3.5 (Task Integration)**: Investigation actions defined
5. **Pattern 3.7 (Sequence Workflows)**: Tasks organized into workflows
6. **Pattern 3.8 (Email Communication Graph)**: Who sent what to whom

### 🔍 The Incident Graph

```
incident
├── sighting_refs → [sighting]
│   ├── sighting_of_ref → indicator (email domain pattern)
│   ├── observed_data_refs → [observed-data]
│   │   └── object_refs → [email-addr, url, email-message]
│   └── extensions → sighting-alert
│       ├── alert_type: "siem"
│       ├── severity: "high"
│       └── source_system: "splunk-prod-01"
├── event_refs → [event]
│   └── sighting_refs → [sighting]
├── task_refs → [task]
│   └── description: "Investigate phishing email"
├── sequence_refs → [event-seq, task-seq]
└── other_object_refs → [indicator, observed-data]
```

Every arrow represents a reference that preserves **provenance** and **context**.

### ? Why This Matters: Incident Reconstitution

Later, you can query the graph to reconstruct the complete story:

**Query**: "Show me the phishing incident"
**Result**: The incident object with all its references

**Query**: "How was this detected?"
**Result**: Follow `sighting_refs` → sighting → `extensions.sighting-alert.alert_type: "siem"`

**Query**: "What evidence was found?"
**Result**: Follow `sighting_refs` → sighting → `observed_data_refs` → observed-data → `object_refs`

**Query**: "What actions were taken?"
**Result**: Follow `task_refs` → tasks with their descriptions and owners

**Query**: "When did this happen?"
**Result**: Follow `event_refs` → events with timestamps

The graph **preserves the complete investigation story** for auditing, learning, and sharing.

### 🚀 Next Steps: Extend the Incident (Step_3)

This incident is based on an **automated SIEM alert**. But 5 hours later, the CEO submits a report: "I received a suspicious email."

In **Step_3**, you'll extend this incident with:
- **Anecdote SCO** (user report) - Pattern 3.9
- **Sighting-anecdote** (human verification)
- **Updated impact** (lower severity since user didn't click) - Pattern 3.10
- **Second event** (user-reported)

The incident will **grow** as new evidence arrives. You'll see how:
- Multiple sightings point to the same incident
- Impact assessments can supersede each other
- Events build a complete timeline

**The incident is alive - it evolves as the investigation progresses.**

### 🎉 Step_2 Complete!

You've created a complete incident investigation from scratch. You now understand:
- How evidence is documented (observables)
- How threats are detected (indicators + sightings)
- How timelines are built (events)
- How responses are planned (tasks + sequences)
- How everything connects (incident container)

**Welcome to advanced STIX incident management!** 🚀

---

**Next**: Open `Step_3_Get the Anecdote.ipynb` to extend this incident with user-reported evidence!